In [ ]:
import os
import sys
import json
import glob
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import seaborn as sns
import networkx as nx
import scipy as sp

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import datetime

sys.path.append("../python/")

from plotutils import *
from metricutils import *

In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", palette="deep", rc=custom_params)

project_root  = "../"

layer = "cnig_provincias"
base_folder = f"TE_{layer}"
base_folder = os.path.join(project_root, f"outputs/{base_folder}")

cases_ds = xr.open_dataset(os.path.join(base_folder, "cases_pop_ds.nc"))
risk_ds = xr.open_dataset(os.path.join(base_folder, "risk_ds.nc"))
df_layer = pd.read_parquet(f"../data/{layer}.parquet")

layer = "abs_09"
base_folder = os.path.join(project_root, f"outputs")
fname = os.path.join(base_folder, f"TE_{layer}/risk_ds.nc")
mobility_ds_cat = xr.load_dataset(fname)
fname = os.path.join(base_folder, f"TE_{layer}/run_w14_d7_R_risk_ij/TE.nc")
TE_cat = xr.load_dataarray(fname)


layer = "zon_bas_13"
base_folder = os.path.join(project_root, f"outputs")
fname = os.path.join(base_folder, f"TE_{layer}/risk_ds.nc")
mobility_ds_mad = xr.load_dataset(fname)
fname = os.path.join(base_folder, f"TE_{layer}/run_w14_d7_R_risk_ij/TE.nc")
TE_mad = xr.load_dataarray(fname)


In [ ]:
mask = (df_layer["ca_name"]=="Cataluña/Catalunya") | (df_layer["ca_name"]=="Comunidad de Madrid")
df_aux = df_layer[mask]

#############################################################################################

total_population = cases_ds["total_population"].loc[df_aux.index, :].to_pandas()
total_population = pd.merge(total_population, df_aux["ca_id"], left_index=True, right_index=True).groupby("ca_id").sum().T
total_population = total_population.mean()

moving_population = cases_ds["moving_population"].loc[df_aux.index, :].to_pandas()
moving_population = pd.merge(moving_population, df_aux["ca_id"], left_index=True, right_index=True).groupby("ca_id").sum().T
moving_population /= total_population.mean()

#############################################################################################

new_cases = cases_ds["new_cases"].loc[df_aux.index, :].to_pandas()
# new_cases = new_cases.rolling(window=3, min_periods=1).mean()

new_cases = pd.merge(new_cases, df_aux["ca_id"], left_index=True, right_index=True).groupby("ca_id").sum().T
new_cases_by_100k = (new_cases / total_population) * 100000
d_new_cases_by_100k_dt = new_cases_by_100k.diff()

active_cases = cases_ds["active_cases"].loc[df_aux.index, :].to_pandas()
active_cases = pd.merge(active_cases, df_aux["ca_id"], left_index=True, right_index=True).groupby("ca_id").sum().T
active_cases_by_100k = (active_cases / total_population) * 100000
dactive_cases_by_100k_dt = active_cases_by_100k.diff()


trips_cat = mobility_ds_cat["trips_ij"].sum(["source", "target"]).to_pandas()
date_range = pd.date_range(trips_cat.index[0], trips_cat.index[-1])
trips_cat = trips_cat.reindex(date_range)
trips_cat.name = "09"
trips_cat /= total_population["09"]

# TE_mad.sum(["source", "target"]).to_pandas()


trips_mad = mobility_ds_mad["trips_ij"].sum(["source", "target"]).to_pandas()
date_range = pd.date_range(trips_mad.index[0], trips_mad.index[-1])
trips_mad = trips_mad.reindex(date_range)
trips_mad.name = "13"
trips_mad /= total_population["13"]

trips = pd.concat([trips_cat, trips_mad], axis=1)
trips_by_pop = (trips / total_population)

#############################################################################################

start_npi = pd.to_datetime("16-10-2020", dayfirst=True)
end_npi   = pd.to_datetime("26-11-2020", dayfirst=True)

start_date = pd.to_datetime("01-03-2020", dayfirst=True)
end_date   = pd.to_datetime("01-02-2021", dayfirst=True)

#############################################################################################

new_cases                = new_cases.loc[start_date:end_date, :]
new_cases_by_100k        = new_cases_by_100k.loc[start_date:end_date, :]
active_cases             = active_cases.loc[start_date:end_date, :]
active_cases_by_100k     = active_cases_by_100k.loc[start_date:end_date, :]
dactive_cases_by_100k_dt = dactive_cases_by_100k_dt.loc[start_date:end_date, :]
trips                    = trips.loc[start_date:end_date, :]
trips_by_pop             = trips

#############################################################################################

ca_id          = "09"
trips_cat.name = "trips"
df_cat         = pd.DataFrame(trips_cat)
df_cat["dI"]   = d_new_cases_by_100k_dt.loc[:, ca_id]
df_cat["I"]    = new_cases_by_100k.loc[:, ca_id]
df_cat["CA"]   = "Catalunya"
DI_cat         = calculate_net_TSxy(TE_cat)
DI_cat         = DI_cat.sum(["source", "target"]).to_pandas()
DI_cat.name    = "DI+"
date_range     = pd.date_range(DI_cat.index[0], DI_cat.index[-1])
DI_cat         = DI_cat.reindex(date_range)
DI_cat_avg     = DI_cat / TE_cat.coords['source'].shape[0]
df_cat         = pd.merge(df_cat, DI_cat_avg, left_index=True, right_index=True)


ca_id          = "13"
trips_mad.name = "trips"
df_mad         = pd.DataFrame(trips_mad)
df_mad["dI"]   = d_new_cases_by_100k_dt.loc[:, ca_id]
df_mad["I"]    = new_cases_by_100k.loc[:, ca_id]
df_mad["CA"]   = "Madrid"
DI_mad         = calculate_net_TSxy(TE_mad)
DI_mad         = DI_mad.sum(["source", "target"]).to_pandas()
DI_mad.name    = "DI+"
date_range     = pd.date_range(DI_mad.index[0], DI_mad.index[-1])
DI_mad         = DI_mad.reindex(date_range)
DI_mad_avg     = DI_mad / TE_mad.coords['source'].shape[0]
df_mad         = pd.merge(df_mad, DI_mad_avg, left_index=True, right_index=True)

In [ ]:
def highlight_period(ax, start, end, ymin=0, ymax=100, alpha=0.15):
    start_num = mdates.date2num(start)
    end_num = mdates.date2num(end)
    rect = patches.Rectangle((start_num, ymin), end_num - start_num, ymax - ymin, edgecolor="black", 
                             facecolor='grey', alpha=alpha)
    ax.add_patch(rect)


plot_label_parms = dict(fontsize=13, verticalalignment='top', fontfamily='sans', weight='bold')
legend_params = dict(loc="upper right", bbox_to_anchor=(0.24, 1.2), framealpha=1)
fig, axes = plt.subplots(3,1, figsize=(12,5), sharex=True, dpi=300)

panel_labels = ["A", "B", "C"]
list_dfs = [trips_by_pop, active_cases_by_100k, dactive_cases_by_100k_dt]
list_ylabels = [r'$M(t)$ by inhabitant', r'$AI(t) by 100K$', r'$\mathit{\Delta}AI^{10d}(t)$ by 100K']
for i,ax in enumerate(axes):
    
    df_data = list_dfs[i]
    ylabel = list_ylabels[i]
    
    ax.plot(df_data["13"], "--", label="Madrid")
    ax.plot(df_data["09"], "r-", label="Catalunya")
    ax.set_ylabel(ylabel, fontsize=12)
    ax.legend(ncol=2, **legend_params)
    highlight_period(ax, start_npi, end_npi, ymin=df_data.min().min(), ymax=df_data.max().max())
    ax.text(-0.025, 1.22, panel_labels[i], transform=ax.transAxes, **plot_label_parms)
    

axes[2].xaxis.set_major_formatter(mdates.DateFormatter('%m/%y'))
axes[2].set_xlim([datetime.date(2020, 3, 1), datetime.date(2021, 2, 1)])
sns.despine(offset=5)
axes[0].spines['bottom'].set_visible(False)
axes[0].xaxis.set_ticks_position('none') 
axes[1].spines['bottom'].set_visible(False)
axes[1].xaxis.set_ticks_position('none') 

fig.tight_layout()
fig.savefig("../figures/supp/FigureS13.tiff", dpi=300)

In [ ]:
def annotate(data, ax, x='trips', y='dI', **kws):
    r, p = sp.stats.pearsonr(data[x], data[y])
    ax.text(.75, .05, f'$R=${r:.2f}\n$p=${p:.1g}',transform=ax.transAxes, fontsize=11)


plot_label_parms = dict(fontsize=12, verticalalignment='top', fontfamily='sans', weight='bold')

xoff = -0.15
yoff = 1.1

    
fig, axes = plt.subplots(2,2, figsize=(8,5.5), dpi=300)

ax = axes[0,0]
data = df_cat.loc[start_date:end_date]

sns.regplot(data, x="trips", y="dI", ax=ax, scatter_kws={'alpha':0.3}, color="r")
annotate(data, ax)
ax.set_xlabel("")
ax.set_ylabel(r"$dI(t)/dt$")
ax.text(xoff, yoff, "A", transform=ax.transAxes, **plot_label_parms)

ax = axes[0,1]
data = df_mad[start_date:end_date]

sns.regplot(data, x="trips", y="dI", ax=ax, scatter_kws={'alpha':0.3})
annotate(data, ax)
ax.set_xlabel("")
ax.set_ylabel(r"")
ax.text(xoff, yoff, "B", transform=ax.transAxes, **plot_label_parms)

ax = axes[1,0]
data = df_cat.loc[start_npi:end_npi]
sns.regplot(data, x="trips", y="dI", ax=ax, scatter_kws={'alpha':0.3}, color="r")
annotate(data, ax)
ax.set_xlabel("Trips by individual")
ax.set_ylabel(r"$dI(t)/dt$")
ax.text(xoff, yoff, "C", transform=ax.transAxes, **plot_label_parms)

ax = axes[1,1]
data = df_mad.loc[start_npi:end_npi]
sns.regplot(data, x="trips", y="dI", ax=ax, scatter_kws={'alpha':0.3})
annotate(data, ax)
ax.set_xlabel("Trips by individual")
ax.set_ylabel(r"")
ax.text(xoff, yoff, "D", transform=ax.transAxes, **plot_label_parms)

fig.savefig("../figures/supp/FigureS14.tiff", dpi=300)

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(8,5.5), dpi=300)

x = "DI+"
y = "dI"

ax = axes[0,0]
data = df_cat.loc[start_date:end_date]

sns.regplot(data, x=x, y=y, ax=ax, scatter_kws={'alpha':0.3}, color="r")
annotate(data, ax, x=x, y=y)
ax.set_xlabel("")
ax.set_ylabel(f"{y}(t)")
ax.text(xoff, yoff, "A", transform=ax.transAxes, **plot_label_parms)

ax = axes[0,1]
data = df_mad[start_date:end_date]

sns.regplot(data, x=x, y=y, ax=ax, scatter_kws={'alpha':0.3})
annotate(data, ax, x=x, y=y)
ax.set_xlabel("")
ax.set_ylabel(r"")
ax.text(xoff, yoff, "B", transform=ax.transAxes, **plot_label_parms)

ax = axes[1,0]
data = df_cat.loc[start_npi:end_npi]
sns.regplot(data, x=x, y=y, ax=ax, scatter_kws={'alpha':0.3}, color="r")
annotate(data, ax, x=x, y=y)
ax.set_xlabel(f"{x}")
ax.set_ylabel(f"{y}(t)")
ax.text(xoff, yoff, "C", transform=ax.transAxes, **plot_label_parms)

ax = axes[1,1]
data = df_mad.loc[start_npi:end_npi]
sns.regplot(data, x=x, y=y, ax=ax, scatter_kws={'alpha':0.3})
annotate(data, ax, x=x, y=y)
ax.set_xlabel(x)
ax.set_ylabel(r"")
ax.text(xoff, yoff, "D", transform=ax.transAxes, **plot_label_parms)

#fig.savefig("../figures/FigureS14.tiff", dpi=300)